In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os
from pymatting import *

In [1]:
!pip install PyMatting

In [5]:
image = load_image("./256img_lst/001.jpg", "RGB")
trimap = load_image("./trimap/001.jpg", "GRAY")
alpha = estimate_alpha_cf(
    image,
    trimap,
    laplacian_kwargs={"epsilon": 1e-6},
    cg_kwargs={"maxiter":2000})

save_image("123.png", alpha)

In [7]:
from pymatting import *
image = load_image("./256img_lst/001.jpg", "RGB")
trimap = load_image("./trimap/001.jpg", "GRAY")
alpha = estimate_alpha_knn(
    image,
    trimap,
    laplacian_kwargs={"n_neighbors": [15, 10]},
    cg_kwargs={"maxiter":2000})
save_image("knn.png", alpha)


In [9]:
image = load_image("./256img_lst/001.jpg", "RGB")
trimap = load_image("./trimap/001.jpg", "GRAY")
alpha = estimate_alpha_lbdm(
    image,
    trimap,
    preconditioner=None,
    laplacian_kwargs={},
    cg_kwargs={})

save_image("lbdm.png", alpha)

In [10]:
image = load_image("./256img_lst/001.jpg", "RGB")
trimap = load_image("./trimap/001.jpg", "GRAY")

alpha = estimate_alpha_lkm(
    image,
    trimap,
    laplacian_kwargs={},
    cg_kwargs={})
save_image("lkm.png", alpha)

In [13]:
image = load_image("./256img_lst/001.jpg", "RGB")
trimap = load_image("./trimap/001.jpg", "GRAY")

alpha = estimate_alpha_rw(
    image,
    trimap,
    laplacian_kwargs={},
    cg_kwargs={})
save_image("rw.png", alpha)

In [12]:
image = load_image("./256img_lst/001.jpg", "RGB")
trimap = load_image("./trimap/001.jpg", "GRAY")

alpha = estimate_alpha_sm(
    image,
    trimap,
    return_foreground_background=False,
    trimap_expansion_radius=10,
    trimap_expansion_threshold=0.02,
    sample_gathering_angles=4,
    sample_gathering_weights=(3.0, 2.0, 1.0, 4.0),
    sample_gathering_Np_radius=1,
    sample_refinement_radius=5,
    local_smoothing_radius1=5,
    local_smoothing_radius2=5,
    local_smoothing_radius3=5,
    local_smoothing_sigma_sq1=None,
    local_smoothing_sigma_sq2=0.1,
    local_smoothing_sigma_sq3=None)
save_image("sm.png", alpha)

In [8]:
import scipy
# height and width of trimap
h, w = trimap.shape[:2]

# calculate laplacian matrix
L = cf_laplacian(image)

# decompose trimap
is_fg, is_bg, is_known, is_unknown = trimap_split(trimap)

# constraint weight
lambda_value = 100.0

# build constraint pixel selection matrix
c = lambda_value * is_known
C = scipy.sparse.diags(c)

# build constraint value vector
b = lambda_value * is_fg

# build linear system
A = L + C

# build ichol preconditioner for faster convergence
A = A.tocsr()
A.sum_duplicates()
M = ichol(A)

# solve linear system with conjugate gradient descent
x = cg(A, b, M=M)

# clip and reshape result vector
alpha = np.clip(x, 0.0, 1.0).reshape(h, w)
save_image("123.png", alpha)